In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#python setup_database.py & python setup_qdrant.py & python setup_elastic.py & python setup_redis.py

In [3]:
import sys
sys.path.append('/app')

In [4]:

from api_client import APIClient
from uuid import UUID
import asyncio
from datetime import datetime, timedelta

In [5]:

client = await APIClient.create(email="dmitrygrankin0@gmail.com")

In [ ]:
client

In [7]:

# Auth examples
#auth_result = await client.google_auth("google_token_here", {"utm_source": "web"})



In [8]:
#token_result = await client.submit_token("918c9f3899d54f3bbcf445cd053c2304")
    


### Contents

In [9]:
contents = await client.get_contents(
    only_archived=False,
    offset=0,
    limit=20
)

In [10]:
contents

{'total': 166,
 'contents': [{'content_id': '0349a69c-8028-49e5-8d20-7906e4e1d5dd',
   'type': 'meeting',
   'text': None,
   'timestamp': '2024-09-05T09:16:22',
   'is_indexed': True,
   'is_owner': True,
   'access_level': 'owner',
   'entities': []},
  {'content_id': '048c81b0-359c-463f-8159-4ccbca8bf185',
   'type': 'meeting',
   'text': None,
   'timestamp': '2025-01-28T10:44:41',
   'is_indexed': True,
   'is_owner': True,
   'access_level': 'owner',
   'entities': []},
  {'content_id': '05016b38-185d-4bb9-9ccf-2352775a4522',
   'type': 'meeting',
   'text': None,
   'timestamp': '2024-11-08T20:33:54',
   'is_indexed': True,
   'is_owner': True,
   'access_level': 'owner',
   'entities': []},
  {'content_id': '07e1c53a-5157-4cfb-9a56-7672ad3482d9',
   'type': 'meeting',
   'text': None,
   'timestamp': '2025-01-14T18:31:55',
   'is_indexed': True,
   'is_owner': True,
   'access_level': 'owner',
   'entities': []},
  {'content_id': '080975d2-7261-4ee1-83d5-556408ff8509',
   'type

In [11]:
# Add new content
content = await client.add_content(
    body="Sample content text",
    content_type="note",
    entities=[{"type": "tag", "name": "test"}]
)


In [12]:
content_id = content["content_id"]

In [13]:
content_id = '0349a69c-8028-49e5-8d20-7906e4e1d5dd'

In [14]:

# Get single content
single_content = await client.get_content(content_id)
single_content

{'content_id': '0349a69c-8028-49e5-8d20-7906e4e1d5dd',
 'type': 'meeting',
 'text': "Meeting Metadata:\nStart Date and Time: 2024-09-05 09:31:44\nSpeakers (Always  reference the names EXACTLY as provided with NO changes letter by letter):\n\nMeeting Transcript:\n\n(0): :  It takes a while, but it works, so...\n(1): :  should be something that's not entirely correct yeah or absolutely incorrect uh no like all the\n(2): :  requests are pending for up to yeah they they resolved like uh four minutes four minutes per",
 'timestamp': '2024-09-05T09:16:22',
 'is_indexed': True,
 'access_level': 'owner',
 'is_owner': True,
 'entities': [],
 'parent_id': None,
 'meta': None}

In [77]:
# Modify content
modified = await client.modify_content(
    content_id,
    body="Updated content text",
    entities=[{"type": "speaker", "name": "Jane Doe"}]
)


In [78]:
modified

{'success': True}

In [79]:
# Get single content
single_content = await client.get_content(content_id)
single_content



{'content_id': 'a7ab7b39-ed6a-429e-988f-00191e477c69',
 'type': 'note',
 'text': 'Updated content text',
 'timestamp': '2025-01-28T19:24:33.833424',
 'is_indexed': False,
 'access_level': 'owner',
 'is_owner': True,
 'entities': [{'name': 'Jane Doe', 'type': 'speaker'}],
 'parent_id': None,
 'meta': None}

In [80]:
archived = await client.archive_content(content_id)

In [81]:
archived

{'success': True}

In [82]:
content_id

UUID('a7ab7b39-ed6a-429e-988f-00191e477c69')

In [83]:
try:
    single_content = await client.get_content(content_id)
except Exception as e:
    print(e)


#OK - content unavailable after archive

Content not found or no access


In [84]:
restored = await client.restore_content(content_id)
single_content = await client.get_content(content_id)
single_content

{'content_id': 'a7ab7b39-ed6a-429e-988f-00191e477c69',
 'type': 'note',
 'text': 'Updated content text',
 'timestamp': '2025-01-28T19:24:33.833424',
 'is_indexed': False,
 'access_level': 'owner',
 'is_owner': True,
 'entities': [{'name': 'Jane Doe', 'type': 'speaker'}],
 'parent_id': None,
 'meta': None}

In [85]:

try:
    deleted = await client.delete_content(content_id)
except Exception as e:
    print(e)
try:
    single_content = await client.get_content(content_id)
except Exception as e:
    print(e)
    
#OK - content unavailable after delete

Content not found or no access


In [86]:

try:
    restored = await client.restore_content(content_id)
except Exception as e:
    print(e)
try:
    single_content = await client.get_content(content_id)
except Exception as e:
    print(e)

#OK - content unavailable after delete

Content not found or not archived


In [87]:
# Add new content
content = await client.add_content(
    body="Sample content text",
    content_type="note",
    entities=[{"type": "speaker", "name": "John Doe"}]
)
content

{'content_id': 'fc55550a-ad9b-46e9-b084-de2daa56e59b'}

In [88]:
# Sharing
share_link = await client.create_share_link(
    access_level="transcript",
    content_ids=[content_id],
    target_email="recipient@example.com",
    expiration_hours=24
)
    


In [89]:
accepted = await client.accept_share_link(
    token=share_link["token"],
    accepting_email="recipient@example.com"
)


In [90]:

# Index content
indexed = await client.index_content(content_id)
indexed



{'status': 'queued', 'message': 'Content has been queued for indexing'}

In [14]:
# Entity management
entities = await client.get_entities("speaker", offset=0, limit=20)
entities


[{'type': 'speaker', 'name': 'John Doe', 'id': 5704},
 {'type': 'speaker', 'name': 'Jane Doe', 'id': 5705}]

In [92]:
# Entity management
entities = await client.get_entities("tag", offset=0, limit=20)
entities


[{'type': 'tag', 'name': 'AI', 'id': 5729},
 {'type': 'tag', 'name': 'test', 'id': 5722}]

In [93]:
# Chat interactions
async for response in client.chat(
    query="What is this document about?",
    content_id=content_id,
    model="gpt-4o-mini",
    temperature=0.7
):
    print(response)



{'chunk': 'Could'}
{'chunk': ' you'}
{'chunk': ' please'}
{'chunk': ' provide'}
{'chunk': ' the'}
{'chunk': ' document'}
{'chunk': ' or'}
{'chunk': ' specify'}
{'chunk': ' its'}
{'chunk': ' content'}
{'chunk': '?'}
{'chunk': ' This'}
{'chunk': ' will'}
{'chunk': ' help'}
{'chunk': ' me'}
{'chunk': ' give'}
{'chunk': ' you'}
{'chunk': ' a'}
{'chunk': ' detailed'}
{'chunk': ' overview'}
{'chunk': ' of'}
{'chunk': ' what'}
{'chunk': ' it'}
{'chunk': ' is'}
{'chunk': ' about'}
{'chunk': '.'}
{'thread_id': '6dc737f7-4cb9-40e0-a947-b871ac3a5dd1', 'content_id': 'a7ab7b39-ed6a-429e-988f-00191e477c69', 'entity_id': None, 'content_ids': None, 'entity_ids': None, 'output': 'Could you please provide the document or specify its content? This will help me give you a detailed overview of what it is about.', 'linked_output': 'Could you please provide the document or specify its content? This will help me give you a detailed overview of what it is about.', 'service_content': {'output': 'Could you pleas

In [94]:
# Edit chat message
async for response in client.edit_chat_message(
    thread_id="thread_123",
    message_index=1,
    new_content="Updated message content",
    model="gpt-4o-mini"
):
    print(response)


{'error': "Edit failed: ChatManager.edit_and_continue() missing 1 required positional argument: 'context_provider'"}


ValueError: Thread not found

In [12]:

# Thread management
threads = await client.get_threads(
 #   content_id=content_id,
    only_archived=False,
    limit=50,
    start_date=datetime.now() - timedelta(days=7)
)


In [13]:
threads

{'total': 3,
 'threads': [{'thread_id': '08bef20f-fccf-4fd9-bdf8-53b25839aae6',
   'thread_name': 'and the last person you mentioned name is?',
   'timestamp': '2025-01-28T19:19:54.658328+00:00',
   'content_id': None,
   'entity_id': None,
   'meta': {'content_ids': ['a0c3f703-2a5e-4ed0-9561-807121516984',
     '6a7287fb-b6ae-46cf-8b99-1ce1bcc81403',
     '580bb613-00de-4c4a-b048-2c2576fcdef0',
     'fb996504-0475-4ee9-a85e-dda4e9f15a4b']},
   'is_archived': False},
  {'thread_id': 'ff7aaa47-23e7-4af7-b871-4528e4272a72',
   'thread_name': 'and the last person you mentioned name is?',
   'timestamp': '2025-01-28T19:14:22.901236+00:00',
   'content_id': None,
   'entity_id': None,
   'meta': {'content_ids': ['a0c3f703-2a5e-4ed0-9561-807121516984',
     '6a7287fb-b6ae-46cf-8b99-1ce1bcc81403',
     '580bb613-00de-4c4a-b048-2c2576fcdef0',
     'fb996504-0475-4ee9-a85e-dda4e9f15a4b']},
   'is_archived': False},
  {'thread_id': '90dce980-5dff-4559-9e25-c59a39f6fce0',
   'thread_name': 'and t

In [ ]:


thread = await client.get_thread("thread_123")
archived_thread = await client.archive_thread("thread_123")
restored_thread = await client.restore_thread("thread_123")
renamed = await client.rename_thread("thread_123", "New Thread Name")
